# 🔍 Fake News Detection with Explainable NLP

This notebook demonstrates the complete pipeline for detecting fake news with interpretable AI explanations.

## Features:
- Text preprocessing and cleaning
- TF-IDF feature extraction
- Multiple ML models (Logistic Regression, Random Forest, SVM)
- LIME explanations for predictions
- Visualization of word importance

## 1. Setup and Imports

In [ ]:
# Install required packages (uncomment if needed)
# !pip install pandas numpy scikit-learn nltk lime shap matplotlib

In [ ]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add project root and src directory to path
# When running from notebooks folder, go up one level
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_path = os.path.join(project_root, 'src')

if src_path not in sys.path:
    sys.path.insert(0, src_path)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Import our modules (ignore Pylance warnings - these resolve at runtime)
from preprocessing import TextPreprocessor, DataLoader, create_sample_dataset  # type: ignore
from feature_engineering import TfidfFeatureExtractor  # type: ignore
from models import ModelTrainer, ModelComparison, print_evaluation_report  # type: ignore
from explainability import LimeExplainer, TextExplainerPipeline, visualize_word_importance  # type: ignore

print(f"✅ Project root: {project_root}")
print("✅ All modules imported successfully!")

## 2. Load and Explore Data

We'll create a sample dataset for demonstration. Replace this with your actual dataset.

In [ ]:
# Create sample dataset for demo
os.makedirs('data', exist_ok=True)
create_sample_dataset('data/sample_news.csv', num_samples=1000)

# Load data
df = pd.read_csv('data/sample_news.csv')
print(f"Dataset shape: {df.shape}")
df.head()

In [ ]:
# Check label distribution
print("Label Distribution:")
print(df['label'].value_counts())

# Visualize
fig, ax = plt.subplots(figsize=(8, 5))
df['label'].value_counts().plot(kind='bar', ax=ax, color=['#e74c3c', '#2ecc71'])
ax.set_title('Label Distribution', fontsize=14)
ax.set_xlabel('Label')
ax.set_ylabel('Count')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

## 3. Text Preprocessing

In [ ]:
# Initialize preprocessor
preprocessor = TextPreprocessor(
    remove_stopwords=True,
    lemmatize=True,
    lowercase=True
)

# Example preprocessing
sample_text = "BREAKING NEWS!!! Scientists discover SHOCKING evidence at https://example.com 😱😱😱"
print(f"Original: {sample_text}")
print(f"Preprocessed: {preprocessor.preprocess(sample_text)}")

In [ ]:
# Preprocess all texts
texts = df['text'].tolist()
labels = df['label'].map({'FAKE': 0, 'REAL': 1}).values

preprocessed_texts = preprocessor.preprocess_batch(texts, verbose=False)
print(f"Preprocessed {len(preprocessed_texts)} texts")

## 4. Feature Extraction (TF-IDF)

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    preprocessed_texts, labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

In [ ]:
# Initialize TF-IDF vectorizer
tfidf = TfidfFeatureExtractor(
    max_features=5000,
    ngram_range=(1, 2),
    min_df=2
)

# Fit and transform
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print(f"TF-IDF feature shape: {X_train_tfidf.shape}")

In [ ]:
# View top features
top_features = tfidf.get_top_features(20)
print("Top 20 TF-IDF features (by IDF score):")
for word, score in top_features:
    print(f"  {word}: {score:.4f}")

## 5. Model Training and Comparison

In [ ]:
# Compare multiple models
comparison = ModelComparison(
    models=['logistic_regression', 'random_forest', 'svm', 'naive_bayes'],
    cv=5
)

results = comparison.compare(X_train_tfidf, y_train)
print("\n" + comparison.summary())

In [ ]:
# Get best model
best_model_type = comparison.get_best_model('accuracy')

In [ ]:
# Visualize comparison results
model_names = []
accuracies = []
f1_scores = []

for model, result in results.items():
    if 'accuracy' in result:
        model_names.append(model)
        accuracies.append(result['accuracy']['mean'])
        f1_scores.append(result['f1']['mean'])

x = np.arange(len(model_names))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, accuracies, width, label='Accuracy', color='#3498db')
bars2 = ax.bar(x + width/2, f1_scores, width, label='F1-Score', color='#2ecc71')

ax.set_ylabel('Score')
ax.set_title('Model Comparison')
ax.set_xticks(x)
ax.set_xticklabels([m.replace('_', ' ').title() for m in model_names], rotation=45)
ax.legend()
ax.set_ylim(0, 1.1)

# Add value labels
for bar in bars1:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height, f'{height:.2f}',
            ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## 6. Train Final Model and Evaluate

In [ ]:
# Train the best model
model = ModelTrainer('logistic_regression', max_iter=1000, C=1.0)
model.train(X_train_tfidf, y_train, feature_names=tfidf.get_feature_names())

In [ ]:
# Evaluate on test set
metrics = model.evaluate(X_test_tfidf, y_test, class_names=['Fake', 'Real'])
print_evaluation_report(metrics, 'Logistic Regression')

In [ ]:
# Plot confusion matrix
from sklearn.metrics import ConfusionMatrixDisplay

fig, ax = plt.subplots(figsize=(8, 6))
ConfusionMatrixDisplay.from_predictions(
    y_test, 
    model.predict(X_test_tfidf),
    display_labels=['Fake', 'Real'],
    cmap='Blues',
    ax=ax
)
ax.set_title('Confusion Matrix', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Feature importance
feature_importance = model.get_feature_importance(top_n=20)

if feature_importance:
    words = [w for w, _ in feature_importance]
    scores = [s for _, s in feature_importance]
    
    fig, ax = plt.subplots(figsize=(10, 8))
    y_pos = np.arange(len(words))
    ax.barh(y_pos, scores, color='#3498db', alpha=0.8)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(words)
    ax.invert_yaxis()
    ax.set_xlabel('Importance Score')
    ax.set_title('Top 20 Most Important Features', fontsize=14)
    plt.tight_layout()
    plt.show()

## 7. Explainability with LIME 🔍

LIME (Local Interpretable Model-Agnostic Explanations) helps us understand **why** the model made a specific prediction.

In [ ]:
# Initialize explainer pipeline
explainer_pipeline = TextExplainerPipeline(
    model=model.model,
    vectorizer=tfidf.vectorizer,
    preprocessor=preprocessor,
    class_names=['Fake', 'Real']
)

In [ ]:
# Example: Explain a fake news sample
fake_news_example = "BREAKING: Government hides shocking conspiracy from public! You won't believe what was revealed!"

result = explainer_pipeline.predict_and_explain(fake_news_example, num_features=10)

print("="*60)
print("FAKE NEWS DETECTION RESULT")
print("="*60)
print(f"\nText: {fake_news_example}")
print(f"\n📊 Prediction: {result['predicted_class']}")
print(f"📈 Confidence: {result['confidence']:.2%}")
print("\n🔍 Why this prediction? (Word Importance):")
for word, score in result['word_importance']:
    direction = "→ Real" if score > 0 else "→ Fake"
    print(f"   '{word}': {score:+.4f} {direction}")

In [ ]:
# Visualize word importance
visualize_word_importance(
    result['word_importance'],
    title=f"Word Importance for Prediction: {result['predicted_class']}",
    figsize=(10, 6)
)

In [ ]:
# Example: Explain a real news sample
real_news_example = "Research study indicates that climate changes may affect economic growth rates over the next decade."

result2 = explainer_pipeline.predict_and_explain(real_news_example, num_features=10)

print("="*60)
print("REAL NEWS DETECTION RESULT")
print("="*60)
print(f"\nText: {real_news_example}")
print(f"\n📊 Prediction: {result2['predicted_class']}")
print(f"📈 Confidence: {result2['confidence']:.2%}")
print("\n🔍 Why this prediction? (Word Importance):")
for word, score in result2['word_importance']:
    direction = "→ Real" if score > 0 else "→ Fake"
    print(f"   '{word}': {score:+.4f} {direction}")

In [ ]:
# Visualize real news explanation
visualize_word_importance(
    result2['word_importance'],
    title=f"Word Importance for Prediction: {result2['predicted_class']}",
    figsize=(10, 6)
)

## 8. Interactive Classification

Try classifying your own text!

In [ ]:
def classify_and_explain(text):
    """Classify text and show explanation."""
    result = explainer_pipeline.predict_and_explain(text, num_features=8)
    
    # Display results
    print("\n" + "="*60)
    print(f"📰 INPUT: {text[:100]}..." if len(text) > 100 else f"📰 INPUT: {text}")
    print("="*60)
    
    # Prediction with emoji
    emoji = "❌" if result['predicted_class'] == 'Fake' else "✅"
    print(f"\n{emoji} Prediction: {result['predicted_class']}")
    print(f"📊 Confidence: {result['confidence']:.1%}")
    
    # Probability bar
    fake_prob = result['probabilities']['Fake']
    real_prob = result['probabilities']['Real']
    print(f"\n   Fake: {'█' * int(fake_prob * 20):<20} {fake_prob:.1%}")
    print(f"   Real: {'█' * int(real_prob * 20):<20} {real_prob:.1%}")
    
    # Word importance
    print("\n🔍 Key Influential Words:")
    for word, score in result['word_importance'][:5]:
        bar = '🟢' if score > 0 else '🔴'
        print(f"   {bar} {word}: {score:+.3f}")
    
    return result

In [ ]:
# Try with different examples
test_texts = [
    "Scientists at MIT publish groundbreaking research on renewable energy solutions.",
    "SHOCKING: This one weird trick will change everything you know about health!",
    "The Federal Reserve announced a 0.25% interest rate increase citing inflation concerns.",
    "You won't BELIEVE what celebrities are hiding from the public!"
]

for text in test_texts:
    classify_and_explain(text)

## 9. Save Model for Production

In [ ]:
# Save model and vectorizer
os.makedirs('models', exist_ok=True)

model.save('models/fake_news_model.pkl')
tfidf.save('models/tfidf_vectorizer.pkl')

print("✅ Model and vectorizer saved successfully!")

## 10. Summary and Next Steps

### What We Accomplished:
1. ✅ Loaded and preprocessed text data
2. ✅ Extracted TF-IDF features
3. ✅ Compared multiple ML models
4. ✅ Trained and evaluated the best model
5. ✅ Implemented LIME explanations
6. ✅ Visualized feature importance

### Future Improvements:
- 🚀 Add BERT embeddings for better accuracy
- 🌍 Implement multilingual support
- 📱 Build web API for real-time detection
- 🔄 Add active learning for continuous improvement